# Welcome to Full Stack Machine Learning's Week 4 Project!

In the final week, you will return to the workflow you built last week on the [taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). 

## Task 1: Deploy the champion
Use what you have learned in the last two weeks to make necessary modifications and to deploy your latest version of the `TaxiFarePrediction` flow to Argo. Use `--branch champion` to denote this deployment as the champion model.

In [4]:
%%writefile ../flows/cloud/taxifare_linear_regression.py
from metaflow import FlowSpec, step, card, conda_base, current, Parameter, Flow, project
from metaflow.cards import Markdown, Table, Image, Artifact

URL = "https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet"
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'

@conda_base(libraries={'pandas': '1.4.2', 'pyarrow': '11.0.0', 'numpy': '1.21.2', 'scikit-learn': '1.1.2'})
@project(name='taxifare')
class TaxiFarePrediction(FlowSpec):

    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):

        # TODO: 
            # Try to complete tasks 2 and 3 with this function doing nothing like it currently is.
            # Understand what is happening.
            # Revisit task 1 and think about what might go in this function.
        obviously_bad_data_filters = [
            df.fare_amount > 0,         # fare_amount in US Dollars
            df.trip_distance <= 100,    # trip_distance in miles
            df.trip_distance > 0,
            df.payment_type <= 2,
            df.passenger_count > 0,
            df.airport_fee > 0,
            df.total_amount > 0
        ]

        for f in obviously_bad_data_filters:
            df = df[f] 
        return df

    @step
    def start(self):

        import pandas as pd
        from sklearn.model_selection import train_test_split

        self.df = self.transform_features(pd.read_parquet(self.data_url))

        # NOTE: we are split into training and validation set in the validation step which uses cross_val_score.
        # This is a simple/naive way to do this, and is meant to keep this example simple, to focus learning on deploying Metaflow flows.
        # In practice, you want split time series data in more sophisticated ways and run backtests. 
        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        self.next(self.linear_model)

    @step
    def linear_model(self):
        "Fit a single variable, linear model to the data."
        from sklearn.linear_model import LinearRegression

        # TODO: Play around with the model if you are feeling it.
        self.model = LinearRegression()

        self.next(self.validate)

    def gather_sibling_flow_run_results(self):

        # storage to populate and feed to a Table in a Metaflow card
        rows = []

        # loop through runs of this flow 
        for run in Flow(self.__class__.__name__):
            if run.id != current.run_id:
                if run.successful:
                    icon = "✅" 
                    msg = "OK"
                    score = str(run.data.scores.mean())
                else:
                    icon = "❌"
                    msg = "Error"
                    score = "NA"
                    for step in run:
                        for task in step:
                            if not task.successful:
                                msg = task.stderr
                row = [Markdown(icon), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(score), Markdown(msg)]
                rows.append(row)
            else:
                rows.append([Markdown("✅"), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(str(self.scores.mean())), Markdown("This run...")])
        return rows
                
    
    @card(type="corise")
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score
        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(Table(self.gather_sibling_flow_run_results(), headers=["Pass/fail", "Run ID", "Created At", "R^2 score", "Stderr"]))
        self.next(self.end)

    @step
    def end(self):
        print("Success!")


if __name__ == "__main__":
    TaxiFarePrediction()

Overwriting ../flows/cloud/taxifare_linear_regression.py


### my note
Champion model deployed in the namespace: `production:mfprj-a3npehmdjhzbo5p3-0-lyrm`

## Task 2: Build the challenger
Develop a second model, by using the same `TaxiFarePrediction` architecture. Then, deploy the flow to Argo as the `--branch challenger`. 
<br>
<br>
Hint: Modify the `linear_model` step. 
<br>
Bonus: Write a paragraph summary of how you developed the second model and tested it before deploying the challenger flow. Let us know in Slack what you found challenging about the task? 

----
### my approach
I wanted to see the difference between linear regressor and some other regressor, and chose Ridge regressor.

### my procedure
1. Copied the TaxiFare baseline code block (as champion) to create another model (as challenger) with differentiating its filename.
2. Tested if the code runs without any errors by executing:
    ```
    python ../flows/cloud/taxifare_different_regression.py --environment=conda run
    ```
3. Deployed the challenger to the cloud:
    ```
    python ../flows/cloud/taxifare_different_regression.py --environment=conda --production --branch challenger argo-workflows create
    python ../flows/cloud/taxifare_different_regression.py --environment=conda --production --branch challenger argo-workflows trigger
    ```

In [5]:
%%writefile ../flows/cloud/taxifare_different_regression.py
from metaflow import FlowSpec, step, card, conda_base, current, Parameter, Flow, project
from metaflow.cards import Markdown, Table, Image, Artifact

URL = "https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet"
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'

@conda_base(libraries={'pandas': '1.4.2', 'pyarrow': '11.0.0', 'numpy': '1.21.2', 'scikit-learn': '1.1.2'})
@project(name='taxifare')
class TaxiFarePrediction(FlowSpec):

    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):

        # TODO: 
            # Try to complete tasks 2 and 3 with this function doing nothing like it currently is.
            # Understand what is happening.
            # Revisit task 1 and think about what might go in this function.
        obviously_bad_data_filters = [
            df.fare_amount > 0,         # fare_amount in US Dollars
            df.trip_distance <= 100,    # trip_distance in miles
            df.trip_distance > 0,
            df.payment_type <= 2,
            df.passenger_count > 0,
            df.airport_fee > 0,
            df.total_amount > 0
        ]

        for f in obviously_bad_data_filters:
            df = df[f] 
        return df

    @step
    def start(self):

        import pandas as pd
        from sklearn.model_selection import train_test_split

        self.df = self.transform_features(pd.read_parquet(self.data_url))

        # NOTE: we are split into training and validation set in the validation step which uses cross_val_score.
        # This is a simple/naive way to do this, and is meant to keep this example simple, to focus learning on deploying Metaflow flows.
        # In practice, you want split time series data in more sophisticated ways and run backtests. 
        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        self.next(self.linear_model)

    @step
    def linear_model(self):
        "Fit a single variable, linear model to the data."
        from sklearn.linear_model import Ridge

        # TODO: Play around with the model if you are feeling it.
        self.model = Ridge()

        self.next(self.validate)

    def gather_sibling_flow_run_results(self):

        # storage to populate and feed to a Table in a Metaflow card
        rows = []

        # loop through runs of this flow 
        for run in Flow(self.__class__.__name__):
            if run.id != current.run_id:
                if run.successful:
                    icon = "✅" 
                    msg = "OK"
                    score = str(run.data.scores.mean())
                else:
                    icon = "❌"
                    msg = "Error"
                    score = "NA"
                    for step in run:
                        for task in step:
                            if not task.successful:
                                msg = task.stderr
                row = [Markdown(icon), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(score), Markdown(msg)]
                rows.append(row)
            else:
                rows.append([Markdown("✅"), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(str(self.scores.mean())), Markdown("This run...")])
        return rows
                
    
    @card(type="corise")
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score
        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(Table(self.gather_sibling_flow_run_results(), headers=["Pass/fail", "Run ID", "Created At", "R^2 score", "Stderr"]))
        self.next(self.end)

    @step
    def end(self):
        print("Success!")


if __name__ == "__main__":
    TaxiFarePrediction()

Overwriting ../flows/cloud/taxifare_different_regression.py


### my note
Challenger model deployed in the namespace: `production:mfprj-axmpwxb4pzis2dci-0-buom`

## Task 3: Analyze the results
Return to this notebook, and read in the results of the challenger and champion flow using the Metaflow Client API.
<br><br>

#### Questions
- Does your model perform better on the metrics you selected? 
- Think about your day job, how would you go about assessing whether to roll forward the production "champion" to your new model? 
    - What gives you confidence one model is better than another?
    - What kinds of information do you need to monitor to get buy-in from stakeholders that model A is preferable to model B?  

### my answers
- _Does your model perform better on the metrics you selected?_
    * No.  The challenger performed slightly better than champion, but it is within an error of fp32.
- _Think about your day job, how would you go about assessing whether to roll forward the production "champion" to your new model?_
    - _What gives you confidence one model is better than another?_
        * The score with new model needs to be clearly better than the champion's.
        * The performance should be better tested by a several different sets of input data. (My comparison did not do this.)
    - _What kinds of information do you need to monitor to get buy-in from stakeholders that model A is preferable to model B?_
        * Model B should bring merits in some target indexes that are defined with the stakeholders comparing to using model A.
        * Model B performs in a stable way with any set of reasonable input data.

In [10]:
from metaflow import Flow, namespace

# these values are unique to your deployment!
CHAMPION_MODEL_NAMESPACE = "production:mfprj-a3npehmdjhzbo5p3-0-lyrm"
CHALLENGER_MODEL_NAMESPACE = "production:mfprj-axmpwxb4pzis2dci-0-buom"

best_score = -1; winner_namespace = None
for n in [CHAMPION_MODEL_NAMESPACE, CHALLENGER_MODEL_NAMESPACE]:
    namespace(n)
    run = Flow('TaxiFarePrediction').latest_successful_run
    acc_score = run.data.scores.mean()
    print(f'{n}\t{acc_score}')
    if acc_score > best_score:
        best_score = acc_score
        winner_namespace = n

print("The winner is the model with R^2 score of {:.4f} in {} namespace.".format(best_score, winner_namespace))

production:mfprj-a3npehmdjhzbo5p3-0-lyrm	0.7731599315109097
production:mfprj-axmpwxb4pzis2dci-0-buom	0.7731599328063418
The winner is the model with R^2 score of 0.7732 in production:mfprj-axmpwxb4pzis2dci-0-buom namespace.


## CONGRATULATIONS! 🎉✨🍾
If you made it this far, you have completed the Full Stack Machine Learning Corise course. 
We are so glad that you chose to learn with us, and hope to see you again in future courses. Stay tuned for more content and come join us in [Slack](http://slack.outerbounds.co/) to keep learning about Metaflow!